In [92]:
import polars as pl
import numpy as np
import yfinance as yf

from hedging_strategy import hedging_strategies, portfolio, risk

In [93]:
spy = yf.download('SPY')[['Adj Close']] * 100
spy_pl = (
    pl.from_pandas(
        spy.reset_index()
    )
    .with_columns(
        pl.col('Date').cast(pl.Date)
    )
    .rename(
        {
            'Adj Close' : 'Holding'
        }
    )
)

[*********************100%***********************]  1 of 1 completed


In [95]:
option_data = (
    pl.read_parquet(
        'Data/SPX_options_1996_2024.parquet'
    )
    .with_columns(
        pl.col('date').cast(pl.Date),
        pl.col('exdate').cast(pl.Date)
    )
)

In [96]:
hedge_date = '2022-12-13'
hedge = hedging_strategies.hedging(
    hedge_date = hedge_date,
    options = option_data,
    close = 4677.03
)

long_put_value = hedge.buy_put()
short_call_value = hedge.short_call()


port = portfolio.portfolio(
    hedge_date = hedge_date,
    holding_value = spy_pl
)

do_nothing = (
    port
    .compute_portfolio()
)

long_put = (
    port
    .compute_portfolio(
        hedge_value = long_put_value
    )
)

short_call = (
    port
    .compute_portfolio(
        hedge_value = short_call_value
    )
)


returns_list = [
    pl.Series(name = 'do_nothing', values = do_nothing['returns'].to_numpy().flatten()[1:]),
    pl.Series(name = 'long_put', values = long_put['returns'].to_numpy().flatten()[1:]),
    pl.Series(name = 'short_call', values = short_call['returns'].to_numpy().flatten()[1:])
]

In [97]:
risk_info = (
    risk.risk(
        returns = returns_list
    )
)

print(risk_info.compute_var(95))

(
    risk_info
    .visualize_returns(
        title = 'Russia Invasion',
        n_rows = 1, 
        height = 600, 
        width = 1500
    )
)

shape: (3, 2)
┌────────────┬──────────┐
│ strategy   ┆ 95_var   │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ do_nothing ┆ 0.014339 │
│ long_put   ┆ 0.044273 │
│ short_call ┆ 0.028345 │
└────────────┴──────────┘
